# Homework 2

**Name:** -- Roberto José --

**e-mail:** -- roberto.jose0745@alumnos.udg.mx --

# Modules

In [ ]:
# Load modules
import numpy as np
import math

#Class Vec2d()

In [ ]:
class Vec2d(object):
    """2D vector class with basic operations"""
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y=None):
        if y is None:
            self.x, self.y = x_or_pair
        else:
            self.x, self.y = x_or_pair, y

    def __add__(self, other):
        return Vec2d(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return Vec2d(self.x - other.x, self.y - other.y)

    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    def rotated(self, angle):
        cos, sin = math.cos(angle), math.sin(angle)
        return Vec2d(self.x * cos - self.y * sin, self.x * sin + self.y * cos)


# Correlated Random Walk (CRW)
A Correlated Random Walk is a movement pattern where each step is influenced by the previous step's direction. Unlike regular random walks where each step is completely random, CRWs tend to keep going in similar directions.

This type of movement is commonly used to model animal foraging behavior, robot path planning, and diffusion processes in physics and biology.

1. Each step:
* Uses the previous direction as a guide
* Adds a small random change in angle
* Moves forward by a fixed distance

2. The math is straightforward:
* Start at point zero (0,0)
* Pick a direction
* For each step:
 * Slightly change direction
 * Move forward
 * Record new position

In [ ]:
# Function to be optimized


In [ ]:
# Run gradient descent algorithm

In [ ]:
# Plot the results